# Combine header for pcm

In [2]:
import csv

In [60]:
vnfs=['bridge','firewall','ndpi_stats',"nf_router",'payload_scan']
folders=["cpu-1","cpu-2","cpu-3","llc-1","vm-1"]

In [7]:
def combine_csv_headers(input_file_path, output_file_path):
    with open(input_file_path, newline='') as input_csvfile:
        reader = csv.reader(input_csvfile)
        first_header = next(reader)
        second_header = next(reader)
        combined_header = [f"{a}-{b}" for a, b in zip(first_header, second_header)]

        with open(output_file_path, 'w', newline='') as output_csvfile:
            writer = csv.writer(output_csvfile)

            # Write the new header
            writer.writerow(combined_header)

            # Write the rest of the lines
            for row in reader:
                writer.writerow(row)

    print(f"new_file：{output_file_path}")



In [11]:
for folder in folders:
    for i in vnfs:
        exp_ls = f'{i}-pcm'
        input_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}.csv'
        output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{i}-pcm_processed.csv'
        combine_csv_headers(input_file_path, output_file_path)



new_file：..\data\pcm\contention\cpu-1\bridge-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-1\firewall-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-1\ndpi_stats-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-1\nf_router-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-1\payload_scan-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-2\bridge-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-2\firewall-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-2\ndpi_stats-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-2\nf_router-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-2\payload_scan-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-3\bridge-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-3\firewall-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-3\ndpi_stats-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-3\nf_router-pcm_processed.csv
new_file：..\data\pcm\contention\cpu-3\payload_scan-pcm_processed.csv
new_file：.

# pcie

In [12]:
exp_ls = f'pcm-pcie'

In [15]:
import csv

def reshape_and_remove_columns(input_file, output_file):
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Extract and modify the header
    original_header = rows[0]
    new_header = [f'skt-0_{h}-total' for h in original_header]+ [f'skt-0_{h}-miss' for h in original_header]  + [f'skt-0_{h}-hit' for h in original_header]+[f'skt-1_{h}-total' for h in original_header]+ [f'skt-1_{h}-miss' for h in original_header]+ [f'skt-1_{h}-hit' for h in original_header]

    # Process the data rows
    reshaped_rows = []
    for i in range(1, len(rows), 7):  # Skip every 7 row (the repeated header)
        row_0 = rows[i]
        if i + 1 < len(rows):  # Check if the next row exists
            row_1 = rows[i+1]
         #   print(row_1)
            row_2 = rows[i+2]
            row_3 = rows[i+3]
            row_4 = rows[i+4]
          #  print(row_4)
            row_5 = rows[i+5]
        #    print(row_5)
            combined_row = row_0 + row_1 + row_2 + row_3 + row_4 + row_5
            reshaped_rows.append(combined_row)

    # Remove columns: Find the index of the columns to be removed
    indices_to_remove = [new_header.index('skt-0_Skt-total'), new_header.index('skt-1_Skt-total'),
                        new_header.index('skt-0_Skt-miss'), new_header.index('skt-1_Skt-miss'),
                        new_header.index('skt-0_Skt-hit'), new_header.index('skt-1_Skt-hit'),]

    # Remove columns: Remove the specified columns from the header and rows
    updated_header = [h for i, h in enumerate(new_header) if i not in indices_to_remove]
    updated_rows = [[item for i, item in enumerate(row) if i not in indices_to_remove] for row in reshaped_rows]

    # Write the processed data to the output file
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(updated_header) 
        writer.writerows(updated_rows) 
    return output_file


In [17]:
folder_1= ["vm-1"]
for folder in folder_1:
    print(folder)
    exp_ls = f'pcm-pcie'
    input_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}.csv'
    output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    result_file =reshape_and_remove_columns(input_file_path, output_file_path)
    print(f"Processed file saved as: {result_file}")

vm-1
Processed file saved as: ..\data\pcm\contention\vm-1\pcm-pcie_processed.csv


In [18]:
import pandas as pd

def clean_numeric_columns(df):
    for col in df.columns:
        df[col] = df[col].astype(str).str.extract(r'(\d+)', expand=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def clean_csv_file(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    cleaned_df = clean_numeric_columns(df)
    cleaned_df.to_csv(output_csv_path, index=False)

# Predefined file paths
input_file = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
output_file =f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'

# Example usage
result_file = clean_csv_file(input_file, output_file)
print(f"Processed file saved as: {result_file}")

Processed file saved as: None


#vm+cpu.etc

In [38]:
import csv

def reshape_and_remove_columns_1(input_file, output_file):
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Extract and modify the header
    original_header = rows[0]
    new_header = [f'skt-0_{h}-total' for h in original_header]+[f'skt-1_{h}-total' for h in original_header]
    print(new_header)
    # Process the data rows
    reshaped_rows = []
    for i in range(1, len(rows), 3):  # Skip every 3 row (the repeated header)
        row_0 = rows[i]
        if i + 1 < len(rows): 
            row_1 = rows[i+1]
            combined_row = row_0 + row_1 
            reshaped_rows.append(combined_row)

    # Remove columns: Find the index of the columns to be removed
    indices_to_remove = [new_header.index('skt-0_Skt-total'), new_header.index('skt-1_Skt-total')]

    # Remove columns: Remove the specified columns from the header and rows
    updated_header = [h for i, h in enumerate(new_header) if i not in indices_to_remove]
    updated_rows = [[item for i, item in enumerate(row) if i not in indices_to_remove] for row in reshaped_rows]

    # Write the processed data to the output file
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(updated_header) 
        writer.writerows(updated_rows) 
    return output_file


In [42]:
folder_2= ["llc-1","cpu-1","cpu-2","cpu-3"]
for folder in folder_2:
    print(folder)
    exp_ls = f'pcm-pcie'
    input_file_path =f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}.csv'
    output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    result_file =reshape_and_remove_columns_1(input_file_path, output_file_path)
    print(f"Processed file saved as: {result_file}")
    
for folder in folder_2:
    print(folder)
    exp_ls = f'pcm-pcie'
    input_file_path =f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    result_file =clean_csv_file(input_file_path, output_file_path)
    print(f"Processed file saved as: {result_file}")

llc-1
['skt-0_Skt-total', 'skt-0_PCIRdCur-total', 'skt-0_RFO-total', 'skt-0_CRd-total', 'skt-0_DRd-total', 'skt-0_ItoM-total', 'skt-0_PRd-total', 'skt-0_WiL-total', 'skt-1_Skt-total', 'skt-1_PCIRdCur-total', 'skt-1_RFO-total', 'skt-1_CRd-total', 'skt-1_DRd-total', 'skt-1_ItoM-total', 'skt-1_PRd-total', 'skt-1_WiL-total']
Processed file saved as: ..\data\pcm\contention\llc-1\pcm-pcie_processed.csv
cpu-1
['skt-0_Skt-total', 'skt-0_PCIRdCur-total', 'skt-0_RFO-total', 'skt-0_CRd-total', 'skt-0_DRd-total', 'skt-0_ItoM-total', 'skt-0_PRd-total', 'skt-0_WiL-total', 'skt-1_Skt-total', 'skt-1_PCIRdCur-total', 'skt-1_RFO-total', 'skt-1_CRd-total', 'skt-1_DRd-total', 'skt-1_ItoM-total', 'skt-1_PRd-total', 'skt-1_WiL-total']
Processed file saved as: ..\data\pcm\contention\cpu-1\pcm-pcie_processed.csv
cpu-2
['skt-0_Skt-total', 'skt-0_PCIRdCur-total', 'skt-0_RFO-total', 'skt-0_CRd-total', 'skt-0_DRd-total', 'skt-0_ItoM-total', 'skt-0_PRd-total', 'skt-0_WiL-total', 'skt-1_Skt-total', 'skt-1_PCIRdCur-

# memory

In [43]:
def combine_csv_headers(input_file_path, output_file_path):
    with open(input_file_path, newline='') as input_csvfile:
        reader = csv.reader(input_csvfile)
        first_header = next(reader)
        second_header = next(reader)
        combined_header = [f"{a}-{b}" for a, b in zip(first_header, second_header)]

        with open(output_file_path, 'w', newline='') as output_csvfile:
            writer = csv.writer(output_csvfile)

            # Write the new header
            writer.writerow(combined_header)

            # Write the rest of the lines
            for row in reader:
                writer.writerow(row)

    print(f"new_file：{output_file_path}")

# Predefined file paths
#exp_ls=f'pcm-memory'
#input_file = f'..\\data\\pcm\\random_input\\exp-9\\{exp_ls}.csv'
#output_file = f'..\\data\\pcm\\random_input\\exp-9\\{exp_ls}_processed.csv'

# Example usage
#result_file = combine_csv_headers(input_file, output_file)
#print(f"Processed file saved as: {result_file}")


In [55]:
folders=["llc-1","vm-1","cpu-1","cpu-2","cpu-3"]
for folder in folders:
    exp_ls = f'pcm-memory'
    input_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}.csv'
    output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    combine_csv_headers(input_file_path, output_file_path)


new_file：..\data\pcm\contention\llc-1\pcm-memory_processed.csv
new_file：..\data\pcm\contention\vm-1\pcm-memory_processed.csv
new_file：..\data\pcm\contention\cpu-1\pcm-memory_processed.csv
new_file：..\data\pcm\contention\cpu-2\pcm-memory_processed.csv
new_file：..\data\pcm\contention\cpu-3\pcm-memory_processed.csv


In [56]:
import csv

def remove_columns_and_empty_values(input_file, output_file):
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    if not rows:
        raise ValueError("CSV file is empty")

    # Find the index of the columns to be removed
    header = rows[0]
    num_columns = len(header)
    indices_to_remove = [i for i, h in enumerate(header) if h.lower() in ['-date', '-time'] or all((len(row) > i and row[i] == '') for row in rows[1:])]

    # Remove the specified columns from the header and rows
    updated_header = [h for i, h in enumerate(header) if i not in indices_to_remove]
    updated_rows = [[item for i, item in enumerate(row[:num_columns]) if i not in indices_to_remove] for row in rows[1:]]

    # Write the updated data to the output file
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(updated_header)  # Write the updated header
        writer.writerows(updated_rows)  # Write the updated data rows

    return output_file

# Predefined file paths
#exp_ls=f'pcm-memory'
#input_file = f'..\\data\\pcm\\const_input\\const-1\\{exp_ls}_processed.csv'
#output_file =f'..\\data\\pcm\\const_input\\const-1\\{exp_ls}_processed.csv'

# Example usage
#result_file = remove_columns_and_empty_values(input_file, output_file)
#print(f"Processed file saved as: {result_file}")

In [57]:
for folder in folders:
    exp_ls = f'pcm-memory'
    input_file_path =  f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    output_file_path =  f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    remove_columns_and_empty_values(input_file_path, output_file_path)
    


# extract traffic and latency information

In [48]:
import pandas as pd

def latency_pre(input_csv, output_csv, delimiter=' '):

    # Read the CSV file
    df = pd.read_csv(input_csv, delimiter=delimiter, header=None)
    df.drop(columns=df.columns[0], inplace=True)
   # df.to_csv(output_csv, index=False, header=False, sep=delimiter)
    df.to_csv(output_csv, index=False, header=['latency'], sep=delimiter)
#input_file = f'..\\data\\pcm\\random_input\\{folder}\\latency.csv'
#output_file =f'..\\data\\pcm\\random_input\\{folder}\\latency_processed.csv'

# Example usage
#result_file = latency_pre(input_file, output_file)

In [59]:
folders=["cpu-3"]
for folder in folders:
    exp_ls = f'latency'
    input_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}.csv'
    output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    latency_pre(input_file_path, output_file_path)

# add header for nf_out csv

In [61]:
import csv

header = ["tag", "instance_id", "service_id", "thread_info.core",
              "rx_pps","tx_pps", "rx", "tx", "act_out", "act_tonf","act_drop", "thread_info.parent",
              "state", "rte_atomic16_read", 
             "rx_drop_rate","tx_drop_rate", "rx_drop","tx_drop", "act_next", 
             "act_buffer","act_returned"]

def nf_out_pre(input_file_path, output_file_path, header):
    with open(input_file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
        
    with open(output_file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        writer.writerows(data)

#input_file = f'..\\data\\pcm\\random_input\\exp-1\\nf_out.csv'
#output_file =f'..\\data\\pcm\\random_input\\exp-1\\nf_out_processed.csv'

# Example usage
result_file = nf_out_pre(input_file, output_file,header)

In [62]:
for folder in folders:
    exp_ls = f'nf_out'
    input_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}.csv'
    output_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    nf_out_pre(input_file_path, output_file_path,header)

In [63]:
import pandas as pd

def process_and_merge_rows(file_path):
    df = pd.read_csv(file_path)
    df.drop(['instance_id', 'service_id'], axis=1, inplace=True)
    tags = ['ndpi_stat', 'router', 'payload_scan', 'bridge', 'firewall']
    features = ["thread_info.core", "rx_pps", "tx_pps", "rx", "tx", "act_out", "act_tonf", "act_drop", "thread_info.parent",
                "state", "rte_atomic16_read", "rx_drop_rate", "tx_drop_rate", "rx_drop", "tx_drop", "act_next", 
                "act_buffer", "act_returned"]
    
    # new dataframe
    new_columns = [f"{tag}-{feature}" for tag in tags for feature in features]
    new_df = pd.DataFrame(columns=new_columns)
    
    # fill data
    for tag in tags:
        tag_df = df[df['tag'] == tag].reset_index(drop=True)
        for feature in features:
            new_df[f"{tag}-{feature}"] = tag_df[feature]
    
    # save processed data
    processed_file_path =  file_path
    new_df.to_csv(processed_file_path, index=False)
    
    return processed_file_path


#file_path = f'..\\data\\pcm\\const_input\\const-3\\nf_out_processed.csv'
#process_and_merge_rows(file_path)


In [65]:
for folder in folders:
    print(folder)
    exp_ls = f'nf_out'
    input_file_path = f'..\\data\\pcm\\contention\\{folder}\\{exp_ls}_processed.csv'
    process_and_merge_rows(input_file_path)
    print(input_file_path)

cpu-1
..\data\pcm\contention\cpu-1\nf_out_processed.csv
cpu-2
..\data\pcm\contention\cpu-2\nf_out_processed.csv
cpu-3
..\data\pcm\contention\cpu-3\nf_out_processed.csv
llc-1
..\data\pcm\contention\llc-1\nf_out_processed.csv
vm-1
..\data\pcm\contention\vm-1\nf_out_processed.csv
